# $KNN$
Алгоритм $K$-ближайших соседей ($KNN$) – это тип управляемого алгоритма *ML*, который может использоваться как для классификации, так и для задач прогнозирования регрессии. Тем не менее, он в основном используется для классификации прогнозирующих проблем в промышленности.

**Алгоритм**

Для классификации каждого из объектов тестовой выборки необходимо последовательно выполнить следующие операции:
- Вычислить расстояние до каждого из объектов обучающей выборки
- Отобрать $k$ объектов обучающей выборки, расстояние до которых минимально
- Класс классифицируемого объекта — это класс, наиболее часто встречающийся среди $k$ ближайших соседей

In [1]:
import pandas as pd
df = pd.read_csv('breast.csv', header = 0)

In [2]:
import numpy as np

# Point - это класс, представляющий точку в декартовой плоскости.
class Point:

    def __init__(self, axis):
        self.axis = np.array(axis)
              
#евклидово расстояние между 2 точками
    def distance(self, other):
        if not isinstance(other, Point):
            other = Point(other)
        return sum((self - other) ** 2) ** 0.5
 
    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.axis + other.axis)
        return Point(self.axis + np.array(other))

    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.axis - other.axis)
        return Point(self.axis - np.array(other))

    def __pow__(self, power, modulo=None):
        if modulo:
            return self.axis ** power % modulo
        return self.axis ** power



In [3]:
from operator import itemgetter

class KNearestNeighbors:
#KNearestNeighbors конструктор
    def __init__(self, k=5):
        self.k = int(k)
        self._fit_data = []

    def fit(self, x, y):
        assert len(x) == len(y)
        self._fit_data = [(Point(coordinates), label) for coordinates, label in zip(x, y)]
#предсказываем выходной класс
    def predict(self, x):
        predicts = []
        for coordinates in x:
            predict_point = Point(coordinates)
            # евклидово расстояние от точки predict_point до всех в self._fit_data
            distances = []
            for data_point, data_label in self._fit_data:
                distances.append((predict_point.distance(data_point), data_label))
            # k точка с меньшими расстояниями
            distances = sorted(distances, key=itemgetter(0))[:self.k]
            predicts.append(max(distances, key=itemgetter(1))[1])

        return predicts

In [4]:
a = KNearestNeighbors()
a.fit(df.iloc[:450, :-1].to_numpy(), df.iloc[:450, -1].to_numpy())


In [5]:
preds = a.predict(df.iloc[450:, :-1].to_numpy())
print('Accuracy:', (sum(preds == df.iloc[450:, -1].to_numpy()) / len(preds)))

Accuracy: 0.907563025210084


Для того, чтобы сравнить мой алгоритм KNN со встроенным. Воспользуемся 'from sklearn.neighbors import KNeighborsClassifier'

In [6]:
from sklearn.neighbors import KNeighborsClassifier
test = KNeighborsClassifier(n_neighbors = 5).fit(df.iloc[:450, :-1], df.iloc[:450, -1:])
print('Accuracy:',(test.score(df.iloc[450:, :-1], df.iloc[450:, -1:])))

Accuracy: 0.8991596638655462


/Users/caesar_julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
